## Step 432 (Day 127) 
### Who am I? What year is it?

Since finals week is now dead and buried, we may continue the exploration with a bit more playfulness.

Based on advice from a colleague (thanks, Dad!), we will now examine whether his die is intentionally or unintentionally weighted. Based on our histogram, it would appear within the realm of likelihood (pun intended) that the die is denser near 5, as Travis rolls a super disproportionate number of 16s.

In [2]:
import pandas as pd 
import numpy as np
import random
import re 
from pprint import pprint

In [42]:
df = pd.read_csv("TravisRollsDataset.csv")

random.seed(15678)
dice_sides = list(range(1,21))
weighted_probs = [0.05]*20
# reduce weight on 5
weighted_probs[4] = weighted_probs[4]*0.5
# increase weight on 16
weighted_probs[15] = round(weighted_probs[15]*1.5, 3)

In [43]:
weighted_samp = list(random.choices(dice_sides, weights=weighted_probs, k=2000))

df2k_new = pd.DataFrame(weighted_samp, columns=["roll"])
df2k_new.index.name = 'id'
df2k_new = df2k_new.reset_index() 

df["basis_type"] = df["basis"].astype(str) + "_" + df["type"].astype(str)

df.head()

,Unnamed: 0,season,episode,roll,mod,basis,type,adv,nat,adv_num,stat,basis_type
0,2,3,1,13,0,wisdom,save,NaN,False,0,WIS,wisdom_save
1,3,3,1,17,0,investigation,check,NaN,False,0,INT,investigation_check
2,4,3,1,4,2,dexterity,save,NaN,False,0,DEX,dexterity_save
3,6,3,2,16,7,melee,attack,NaN,False,0,ATTACK,melee_attack
4,7,3,2,16,7,melee,attack,advantage,False,1,ATTACK,melee_attack


In [46]:
def randassign(x, givdict):
    r = random.random()
    for k in givdict.keys():
        if r <= k:
            return givdict[k]
    return pd.NaN

for c in ["basis_type", "adv", "stat"]:
    tmpdf = pd.DataFrame(df[c].value_counts()).reset_index()
    tmpdf['pct'] = tmpdf[c] / sum(tmpdf[c])
    cum1cumall = dict(tmpdf[["pct","index"]].values)
#     pprint(cum1cumall)
#     print("\n")
    tmpdf['cumsum'] = tmpdf.pct.cumsum()
    cumkeys = dict(tmpdf[["cumsum","index"]].values)
#     pprint(cumkeys)
#     print("\n")
    df2k_new[c] = df2k_new.roll.apply(randassign, args=(cumkeys,))

df2k_new[['basis','type']] = df2k_new.basis_type.str.split("_",expand=True) 

def adv_roll(x):
    r = random.randint(1,20)
    if x['adv'] == "advantage":
        return max(x['roll'], r)
    if x['adv'] == "disadvantage":
        return min(x['roll'], r)
    return x['roll']

df2k_new['roll'] = df2k_new.apply(adv_roll, axis=1)

df2k_new = df2k_new.drop(['basis_type'], axis=1)

# df2k_new

df2k_new.iloc[:, 1:]

,roll,adv,stat,basis,type
0,12,advantage,ATTACK,melee,attack
1,16,advantage,INIT,perception,check
2,16,advantage,WIS,strength,save
3,16,advantage,DEX,melee,attack
4,2,disadvantage,ATTACK,melee,attack
...,...,...,...,...,...
1995,16,advantage,CHA,ranged,attack
1996,20,advantage,STR,strength,check
1997,13,advantage,DEX,strength,check
1998,3,disadvantage,ATTACK,melee,attack


In [47]:
df2k_new.to_csv(r"nerd_shit1.csv", index=False)